Let us consider a problem:
We have an Observation that we would like to translate to DataFrame.

First, we initialize the needed classes.

In [1]:
from fhir_pyrate import Pirate

# # Init authentication for your FHIR server
# # Either authenticate with a token
# import os
# from dotenv import load_dotenv
#
# load_dotenv()
# # I have stored these variables in a .env file
# auth = Ahoy(
#     auth_type="token",
#     auth_method="env",
#     auth_url=os.environ["BASIC_AUTH"],
#     refresh_url=os.environ["REFRESH_AUTH"],
#     # token = ... # The token can also be passed as a variable
# )
# # Or with BasicAuth
# auth = Ahoy(
#     auth_type="BasicAuth",
#     auth_method="env", # The variables FHIR_USER and FHIR_PASSWORD have your authentication data
#     # auth_method="password", # You can also specify username and password
#     # username="abc",
# )
# Init search class
search = Pirate(
    auth=None,
    base_url="http://hapi.fhir.org/baseDstu2",
    print_request_url=True,  # Set it to true to get the URL calls
    num_processes=1,
)

We can now start with our actual query. Let's assume, for this very simple case, that we know the
 Observation ID. Then, we just need to call the following function with the Observation resource
 and the request parameters to obtain our result.

`query_to_dataframe` is a wrapper function that downloads a list of bundles using any function specified in `bundles_function`, and then uses a process function to build a DataFrame with the requested information. The default processing function returns the entire structure of the resource.


In [2]:
observation_all = search.query_to_dataframe(  # Wrapper function
    bundles_function=search.steal_bundles,  # Non-parallel function that iterates through the bundles and returns them
    resource_type="Observation",
    request_params={
        "_count": 1,
        "_id": "86092",
    },
)
observation_all

http://hapi.fhir.org/baseDstu2/Observation?_count=1&_id=86092


Query: 1it [00:00, 5377.31it/s]


,resourceType,id,meta_versionId,meta_lastUpdated,status,category_coding_0_system,category_coding_0_code,code_coding_0_system,code_coding_0_code,code_coding_0_display,code_text,subject_reference,encounter_reference,effectiveDateTime,issued,valueQuantity_value,valueQuantity_unit,valueQuantity_system,valueQuantity_code
0,Observation,86092,1,2018-11-19T12:59:31.238+00:00,final,http://hl7.org/fhir/observation-category,vital-signs,http://loinc.org,29463-7,Body Weight,Body Weight,Patient/86079,Encounter/86090,2011-03-10T20:47:29-05:00,2011-03-10T20:47:29-05:00,6.079781,kg,http://unitsofmeasure.org/,kg


And there it is, the JSON has been now stored into a DataFrame. As you see, the amount of
information is a bit much, there are things that we do not need at all.
Let us assume that we are only interested in the FHIR patient ID, the date and the values. Since
these values are quite easy to get, we can use the `fhir_paths` parameter.
Remember to install [fhirpath-py](https://github.com/beda-software/fhirpath-py) to use the
`fhir_paths` parameter.

In [3]:
observation_tnm = search.query_to_dataframe(
    bundles_function=search.steal_bundles,
    resource_type="Observation",
    request_params={
        "_count": 1,
        "_id": "86092",
    },
    fhir_paths=[
        "id",
        "effectiveDateTime",
        "valueQuantity.value",
        "valueQuantity.unit",
        "subject.reference",
    ],
)
observation_tnm

http://hapi.fhir.org/baseDstu2/Observation?_count=1&_id=86092


Query: 1it [00:00, 21845.33it/s]


,id,effectiveDateTime,valueQuantity.value,valueQuantity.unit,subject.reference
0,86092,2011-03-10T20:47:29-05:00,6.079781,kg,Patient/86079


And there it is, our patient with the values we wanted!